In [ ]:
import sys

sys.path.insert(0, "/home/haridas/projects/opensource/automl/efficientdet/")

In [ ]:
import numpy as np
import pandas as pd
import os

from PIL import Image

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import tensorflow as tf

from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [ ]:
# print_tensors_in_checkpoint_file(file_name="/home/haridas/projects/opensource/automl/efficientdet/efficientdet-d1/", tensor_name="resample_p1", all_tensors=True)

## TFRecord analysis

In [ ]:
tfr_file = "/home/haridas/projects/opensource/automl/efficientdet/data/val/val-00000-of-00001.tfrecord"

In [ ]:
raw_dataset = tf.data.TFRecordDataset(tfr_file)

In [ ]:
raw_record = raw_dataset.take(1)

In [ ]:
example = tf.train.Example()

In [ ]:
# example.ParseFromString(raw_record.numpy())

## Model Inference Experiment

In [ ]:
min_score_threh = 0.35
max_boxes_to_draw = 100
line_thickness = 2

In [ ]:
img_path = "/home/haridas/projects/mystique/data/templates_test_data/3.png"

In [ ]:
img = Image.open(img_path)
img.size

In [ ]:
# !python /home/haridas/projects/opensource/automl/efficientdet/model_inspect.py --help

# tf.estimator.tpu.RunConfig(
#     model_dir="path",
#     evaluation_master="",
#     cluster=None
# )

# References

1. [https://ml6.eu/a-practical-guide-to-fine-tuning-efficientdet-for-object-detection/](https://ml6.eu/a-practical-guide-to-fine-tuning-efficientdet-for-object-detection/)